In [1]:
import os
from pdf2image import convert_from_path
import imageio
import re

In [2]:
def natural_sort_key(s):
    """
    Sort strings with numbers in natural order.
    So reconstruction_epoch_10.pdf comes after reconstruction_epoch_2.pdf
    """
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(r'(\d+)', s)]

def create_looping_gif_from_pdfs(pdf_dir, output_gif_path, num_files=15, duration=0.5):
    """
    Convert first N PDFs to a looping GIF.
    Specifically handles files named 'reconstruction_epoch_X.pdf'
    
    Args:
        pdf_dir (str): Directory containing PDF files
        output_gif_path (str): Path where the output GIF will be saved
        num_files (int): Number of files to process
        duration (float): Duration for each frame in seconds
    """
    # Get list of PDF files in directory matching the pattern
    pdf_files = [f for f in os.listdir(pdf_dir) 
                 if f.startswith('reconstruction_epoch_') and f.endswith('.pdf')]
    
    # Sort files naturally so numbers are in correct order
    pdf_files.sort(key=natural_sort_key)
    
    if not pdf_files:
        raise Exception("No matching PDF files found in the specified directory")
    
    # Take only the first num_files
    pdf_files = pdf_files[:num_files]
    
    # Create a temporary directory for images
    temp_dir = "temp_images"
    os.makedirs(temp_dir, exist_ok=True)
    
    images = []
    try:
        # Convert each PDF to image and add to list
        total_files = len(pdf_files)
        for i, pdf_file in enumerate(pdf_files):
            print(f"Converting file {i+1}/{total_files}: {pdf_file}")
            # Convert PDF to image
            full_path = os.path.join(pdf_dir, pdf_file)
            pages = convert_from_path(full_path)
            # We'll use only the first page of each PDF
            image_path = os.path.join(temp_dir, f"{pdf_file}.png")
            pages[0].save(image_path, 'PNG')
            images.append(imageio.imread(image_path))
        
        # Create looping GIF
        print("Creating GIF...")
        # The loop parameter is set to 0 for infinite looping
        imageio.mimsave(output_gif_path, images, duration=duration, loop=0)
        print(f"Looping GIF saved to {output_gif_path}")
        
    finally:
        # Clean up temporary files
        if os.path.exists(temp_dir):
            for file in os.listdir(temp_dir):
                os.remove(os.path.join(temp_dir, file))
            os.rmdir(temp_dir)

In [ ]:
pdf_dir = "test_try/fixed_samples_progress"
output_gif = "reconstruction_progress_loop.gif"
create_looping_gif_from_pdfs(
    pdf_dir=pdf_dir,
    output_gif_path=output_gif,
    num_files=15,  # Only take first 15 files
    duration=0.5   # Half second per frame
)